# SVM

In [ ]:
import sys; sys.path.append('../../')
from DataPreparing.data_preparing import read_data
from Evaluation.evaluate import *
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE,SMOTENC,SMOTEN,BorderlineSMOTE
from imblearn.under_sampling import NearMiss,RandomUnderSampler
from sklearn.model_selection import train_test_split


# Read Data

In [7]:
X_train, X_val, y_train, y_val =read_data("../../DataFiles/191.csv")

# Initial Fit

In [14]:
%%time
clf = svm.SVC( kernel='linear')
clf.fit(X_train, y_train)
evaluate(clf, X_val,y_val , clf.predict(X_train), y_train)

## Under Sampling

# Hyper parameter Search

# Recursive Feature Elimination 

## Validation Curve (maybe)